In [ ]:
# Author: Yanyu (Claudia) Cheng
# This is the modular, production-ready version of the analysis script
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest

# Global plot configuration
sns.set_theme(style="whitegrid")
CONFIG = {
    "time_cols": ["year", "month", "day", "hour"],
    "features": ["PM2.5", "PM10", "TEMP", "PRES", "DEWP"],
}

def ingest_and_process_data(file_path):
    # Ingests raw sensor data, parses timestamps, and aggregates to daily mean
    try:
        print(f"Loading data from: {file_path}")
        df = pd.read_csv(file_path)
        
        # Validate columns exist before processing
        required_cols = CONFIG['time_cols'] + CONFIG['features']
        if not set(required_cols).issubset(df.columns):
            missing = set(required_cols) - set(df.columns)
            raise ValueError(f"Missing required columns: {missing}")

        # Parse datetime
        df['timestamp'] = pd.to_datetime(df[CONFIG['time_cols']])
        df.set_index('timestamp', inplace=True)
        
        # Select and clean features, using .copy() ensures it doesn't trigger SettingWithCopy warnings
        df_clean = df[CONFIG['features']].apply(pd.to_numeric, errors='coerce').dropna()
        
        # Report data loss
        initial_rows = len(df)
        clean_rows = len(df_clean)
        print(f"Data Cleaning: Dropped {initial_rows - clean_rows} rows ({(initial_rows - clean_rows)/initial_rows:.1%} loss)")

        # Downsample to daily mean
        df_daily = df_clean.resample('D').mean()
        return df_daily
    
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return None
    except Exception as e:
        print(f"Critical Error during ingestion: {e}")
        return None

def detect_anomalies(df_in, contamination=0.05):
    # Trains an Isolation Forest to flag multivariate anomalies
    df = df_in.copy()

    # Contamination set to 5% based on reasonable starting assumption sensor failure rate 
    model = IsolationForest(contamination=contamination, random_state=42)
    
    df['anomaly_score'] = model.fit_predict(df)
    # Isolation Forest returns -1 for anomalies, 1 for normal
    df['is_anomaly'] = df['anomaly_score'] == -1
    
    return df

def visualize_results(df):
    """Generates diagnostic plots for the anomaly detection."""
    
    # Plot 1: Time Series with highlighted anomalies
    plt.figure(figsize=(12, 5))
    sns.lineplot(data=df, x=df.index, y='PM2.5', label='PM2.5 Conc.', alpha=0.6)
    
    anomalies = df[df['is_anomaly']]
    plt.scatter(anomalies.index, anomalies['PM2.5'], color='red', label='Multivariate Anomaly', s=30, marker='x', zorder=5)
    
    plt.title('PM2.5 Time Series with Multivariate Anomalies (Isolation Forest)')
    plt.ylabel('Concentration (ug/m3)')
    plt.legend()
    plt.tight_layout()
    plt.show()

    # Plot 2: Correlation Heatmap
    plt.figure(figsize=(6, 5))
    sns.heatmap(df[CONFIG['features']].corr(), annot=True, cmap='coolwarm', fmt=".2f", vmin=-1, vmax=1)
    plt.title('Feature Correlation Matrix')
    plt.tight_layout()
    plt.show()

# Main Execution Block
def run_pipeline(file_path='../data/raw_data.csv'):
    
    print("--- Starting Pipeline ---")
    df_processed = ingest_and_process_data(file_path)
    
    if df_processed is not None:
        # Inspect Shape
        print(f"Dataset Shape: {df_processed.shape}")
        
        # Model Training
        print("\n--- Training Model ---")
        df_results = detect_anomalies(df_processed)
        
        n_anomalies = df_results['is_anomaly'].sum()
        print(f"Anomalies Detected: {n_anomalies} ({n_anomalies/len(df_results):.1%})")
        
        # Results Preview Table
        print("\nResults Preview (Head):")
        preview = df_results[['PM2.5', 'TEMP', 'is_anomaly']].head().reset_index()
        preview = preview.rename(columns={preview.columns[0]: 'date'})
        preview = preview.round({'PM2.5':4, 'TEMP':4})
        print(preview.to_string(index=False))


        # Visualize
        print("\n--- Generating Visuals ---")
        visualize_results(df_results)
    else:
        raise FileNotFoundError(f"Could not load data from: {file_path}")

    return df_results


In [ ]:
_ = run_pipeline()